In [1]:
import matplotlib

matplotlib.use('Agg')
import numpy as np
from keras import backend as K
from keras import utils as keras_utils
from keras import optimizers
from keras import datasets
from tqdm import tqdm

import cdcgan_models, cdcgan_utils

Using TensorFlow backend.


In [2]:
BATCH_SIZE = 128
EPOCHS = 1

In [3]:
(X_train, y_train), (_, _) = datasets.mnist.load_data()
X_train = cdcgan_utils.transform_images(X_train)
X_train = X_train[:, :, :, None]

y_train = keras_utils.to_categorical(y_train, 100)

In [4]:
print("Generator:")
G = cdcgan_models.generator_model()
G.summary()

Generator:
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1024)         103424      input_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 1024)         103424      input_2[0][0]                    
__________________________________________________________________________________________________

In [5]:
print("Discriminator:")
D = cdcgan_models.discriminator_model()
D.summary()

Discriminator:
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 28, 28, 64)   1664        input_3[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       multiple             0           conv2d_3[0][0]                   
                                                                 conv2d_4[0][0]                   
                                                                 dense_5[0][0]                    
                                                                 dense_7[0][0]                

In [6]:
print("Combined:")
GD = cdcgan_models.generator_containing_discriminator(G, D)
GD.summary()

Combined:
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 28, 28, 1)    13525889    input_5[0][0]                    
                                                                 input_6[0][0]                    
__________________________________________________________________________________________________
model_2 (Model)                 (None, 1)            10158465    model_1[1][0]                    


In [7]:
optimizer = optimizers.Adam(0.0002, 0.5)

In [8]:
G.compile(loss='binary_crossentropy', optimizer=optimizer)
GD.compile(loss='binary_crossentropy', optimizer=optimizer)
D.trainable = True
D.compile(loss='binary_crossentropy', optimizer=optimizer)

In [9]:
iteration = 0

nb_of_iterations_per_epoch = int(X_train.shape[0] / BATCH_SIZE)
print("Number of iterations per epoch: {0}".format(nb_of_iterations_per_epoch))

Number of iterations per epoch: 468


In [10]:
for epoch in range(EPOCHS):
    pbar = tqdm(desc="Epoch: {0}".format(epoch), total=X_train.shape[0])

    g_losses_for_epoch = []
    d_losses_for_epoch = []

    for i in range(nb_of_iterations_per_epoch):
        noise = cdcgan_utils.generate_noise((BATCH_SIZE, 100))

        image_batch = X_train[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
        label_batch = y_train[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]

        generated_images = G.predict([noise, label_batch], verbose=0)

        if i % 20 == 0:
            image_grid = cdcgan_utils.generate_mnist_image_grid(G,
                                                                title="Epoch {0}, iteration {1}".format(epoch,
                                                                                                        iteration))
            cdcgan_utils.save_generated_image(image_grid, epoch, i, "../images/generated_mnist_images_per_iteration")

        X = np.concatenate((image_batch, generated_images))
        y = [1] * BATCH_SIZE + [0] * BATCH_SIZE
        label_batches_for_discriminator = np.concatenate((label_batch, label_batch))

        D_loss = D.train_on_batch([X, label_batches_for_discriminator], y)
        d_losses_for_epoch.append(D_loss)

        noise = cdcgan_utils.generate_noise((BATCH_SIZE, 100))
        D.trainable = False
        G_loss = GD.train_on_batch([noise, label_batch], [1] * BATCH_SIZE)
        D.trainable = True
        g_losses_for_epoch.append(G_loss)

        pbar.update(BATCH_SIZE)

        iteration += 1

    # Save a generated image for every epoch
    image_grid = cdcgan_utils.generate_mnist_image_grid(G, title="Epoch {0}".format(epoch))
    cdcgan_utils.save_generated_image(image_grid, epoch, 0, "../images/generated_mnist_images_per_epoch")

    pbar.close()
    print("D loss: {0}, G loss: {1}".format(np.mean(d_losses_for_epoch), np.mean(g_losses_for_epoch)))

    G.save_weights("../models/generator_mnist.h5")
    D.save_weights("../models/discriminator_mnist.h5")

Epoch: 0: 100%|█████████▉| 59904/60000 [18:28<00:01, 54.99it/s]


D loss: 0.5623317360877991, G loss: 1.391081690788269


OSError: Unable to create file (unable to open file: name = 'models/weights/generator.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 602)

In [12]:
G.save_weights("../models/generator_mnist.h5")
D.save_weights("../models/discriminator_mnist.h5")